In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from albumentations import (
    convert_bbox_to_albumentations,convert_bboxes_to_albumentations
)

# Filtrado del conjunto de entrenamiento

In [2]:
dataDir='/home/sergio/Documents/COCODataset'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
print(annFile)

/home/sergio/Documents/COCODataset/annotations/instances_train2017.json


In [3]:
from pycocotools.coco import COCO
import numpy as np
#import skimage.io as io
import matplotlib.pyplot as plt
#import pylab

# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=8.31s)
creating index...
index created!


In [4]:
catIds = coco.getCatIds(catNms=['person']);
imgIds = coco.getImgIds(catIds=catIds );

In [5]:
aux = coco.loadImgs(imgIds)
real_names =[]
for i in aux:
    real_names.append(i["file_name"])
print(len(real_names))
print(real_names[:12])

64115
['000000262145.jpg', '000000262146.jpg', '000000524291.jpg', '000000262148.jpg', '000000393223.jpg', '000000393224.jpg', '000000524297.jpg', '000000393227.jpg', '000000131084.jpg', '000000393230.jpg', '000000262161.jpg', '000000131089.jpg']


In [10]:
import os
os.chdir("/home/sergio/Documents/COCODataset")
os.mkdir("pedestrian_dataset_train")

In [11]:
from shutil import copyfile

def get_class_images(SOURCE, NEW_LOC,CLASS_NAMES):

    names_list =os.listdir(SOURCE)

    for i in names_list:
        if i in CLASS_NAMES:
                copyfile(SOURCE + "/" + i, NEW_LOC + "/" + i)

#dict_pedestrian = get_pedestrian(data) 
#pedestrian_names = list(dict_pedestrian.keys())
#print(len(pedestrian_names))

In [12]:
#get_class_images("/home/sergio/Documents/COCODataset/train2017","/home/sergio/Documents/COCODataset/pedestrian_dataset_train",real_names)

# Filtrado de imágenes de validación

In [13]:
dataDir='/home/sergio/Documents/COCODataset'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
print(annFile)

/home/sergio/Documents/COCODataset/annotations/instances_val2017.json


In [14]:
from pycocotools.coco import COCO
import numpy as np
#import skimage.io as io
import matplotlib.pyplot as plt
#import pylab

# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


In [15]:
catIds = coco.getCatIds(catNms=['person']);
imgIds = coco.getImgIds(catIds=catIds );

In [16]:
aux = coco.loadImgs(imgIds)
real_names =[]
for i in aux:
    real_names.append(i["file_name"])
print(len(real_names))
print(real_names[:12])

2693
['000000532481.jpg', '000000458755.jpg', '000000385029.jpg', '000000311303.jpg', '000000393226.jpg', '000000532493.jpg', '000000008211.jpg', '000000327701.jpg', '000000188439.jpg', '000000434204.jpg', '000000401446.jpg', '000000213033.jpg']


In [18]:
import os
os.chdir('/home/sergio/Documents/COCODataset')
os.mkdir("pedestrian_dataset_val")

In [19]:
#get_class_images("/home/sergio/Documents/COCODataset/val2017","/home/sergio/Documents/COCODataset/pedestrian_dataset_val",real_names)

## Creating Metadata train CSV

In [6]:
dataDir='/home/sergio/Documents/COCODataset'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
print(annFile)
coco=COCO(annFile)

/home/sergio/Documents/COCODataset/annotations/instances_train2017.json
loading annotations into memory...
Done (t=8.60s)
creating index...
index created!


In [7]:
catIds = coco.getCatIds(catNms=['person']); #ID de la categoría persona
imgIds = coco.getImgIds(catIds=catIds ); # IDS de las imagenes con la categoria persona
annIds = coco.getAnnIds(imgIds=imgIds, catIds=catIds) #Anotaciones de las imágenes con la categoría persona
img_metadata = coco.loadImgs(imgIds) # Metadatos de las images de la categoría persona
img_annotations = coco.loadAnns(annIds) # Anotaciones de las instancias de la base de datos

In [8]:
img_metadata[0]

{'license': 2,
 'file_name': '000000262145.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000262145.jpg',
 'height': 427,
 'width': 640,
 'date_captured': '2013-11-20 02:07:55',
 'flickr_url': 'http://farm8.staticflickr.com/7187/6967031859_5f08387bde_z.jpg',
 'id': 262145}

In [9]:
id_to_file_name={}

for i in img_metadata:
    id_ = str(i["id"])
    file_name = i["file_name"]
    id_to_file_name[id_] = file_name

In [10]:
len(id_to_file_name)

64115

In [11]:
img_metadata_dict = {}

for i in img_metadata:
    img_metadata_dict[i["file_name"]] = {"width":i["width"],"height":i["height"]}

In [12]:
metadata_df = pd.DataFrame(img_metadata_dict).T

In [13]:
metadata_df.head()

,width,height
000000262145.jpg,640,427
000000262146.jpg,480,640
000000524291.jpg,640,426
000000262148.jpg,640,512
000000393223.jpg,640,480


In [14]:
os.chdir("/home/sergio/Documents/COCODataset")
metadata_df.to_csv("COCO_metadata_train.csv",index=True)

In [15]:
len(img_metadata_dict)

64115

## Creating Annotations COCO train CSV

In [16]:
#(img_annotations[0])

In [17]:
dict_annotations = {}
num_instance = 0

#VER COMO SE GUARDARON LAS COORDENADAS DEL VOC DATASET, QUE CORDENADA (X,Y) SON

for index,j in enumerate(img_annotations):
    #print(j)
    #print(id_to_file_name[str(j["image_id"])])
    
    height = metadata_df.loc[id_to_file_name[str(j["image_id"])],"height"]
    #print(height)
    width = metadata_df.loc[id_to_file_name[str(j["image_id"])],"width"]
    #print(width)
    #print(j["bbox"])
    if j["bbox"][2]>1 and j["bbox"][3]>1: 
        [xmin,ymin,xmax,ymax]=list(convert_bbox_to_albumentations((j["bbox"][0],j["bbox"][1],j["bbox"][2],j["bbox"][3]),source_format="coco",check_validity=True,rows = height,cols = width))
    
        dict_annotations[index] = {"file_name":id_to_file_name[str(j["image_id"])],
                                      "xmin":xmin,
                                      "ymin":ymin,
                                      "xmax":xmax,
                                     "ymax": ymax}

In [18]:
annotations_df = pd.DataFrame(dict_annotations).T

In [19]:
annotations_df.head()

,file_name,xmin,ymin,xmax,ymax
0,000000262145.jpg,0.604688,0.166511,0.83125,0.920609
1,000000262145.jpg,0.772359,0.213208,0.790641,0.251054
2,000000262145.jpg,0.441203,0.203255,0.491453,0.430468
3,000000262145.jpg,0.822141,0.103372,0.963547,0.543653
4,000000262145.jpg,0.767984,0.190141,0.831203,0.39


In [20]:
annotations_df.shape

(262461, 5)

In [21]:
os.chdir("/home/sergio/Documents/COCODataset")
annotations_df.to_csv("COCO_annotations_train.csv",index=True)

## Creating metadata validation COCO

In [22]:
dataDir='/home/sergio/Documents/COCODataset'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
print(annFile)
coco  =COCO(annFile)

/home/sergio/Documents/COCODataset/annotations/instances_val2017.json
loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [23]:
catIds = coco.getCatIds(catNms=['person']); #ID de la categoría persona
imgIds = coco.getImgIds(catIds=catIds ); # IDS de las imagenes con la categoria persona
annIds = coco.getAnnIds(imgIds=imgIds, catIds=catIds) #Anotaciones de las imágenes con la categoría persona
img_metadata = coco.loadImgs(imgIds) # Metadatos de las images de la categoría persona
img_annotations = coco.loadAnns(annIds) # Anotaciones de las instancias de la base de datos

In [24]:
id_to_file_name={}

for i in img_metadata:
    id_ = i["id"]
    #print(id_)
    file_name = i["file_name"]
    id_to_file_name[id_] = file_name

In [25]:
img_metadata_dict = {}

for i in img_metadata:
    img_metadata_dict[i["file_name"]] = {"width":i["width"],"height":i["height"]}

In [26]:
metadata_df = pd.DataFrame(img_metadata_dict).T

In [27]:
metadata_df.head()

,width,height
000000532481.jpg,640,426
000000458755.jpg,640,480
000000385029.jpg,640,480
000000311303.jpg,640,427
000000393226.jpg,640,480


In [28]:
os.chdir("/home/sergio/Documents/COCODataset")
metadata_df.to_csv("COCO_metadata_val.csv",index=True)

## Creating annotatios val COCO

In [29]:
id_to_file_name={}

for i in img_metadata:
    id_ = str(i["id"])
    file_name = i["file_name"]
    id_to_file_name[id_] = file_name

In [30]:
dict_annotations = {}
num_instance = 0

#VER COMO SE GUARDARON LAS COORDENADAS DEL VOC DATASET, QUE CORDENADA (X,Y) SON

for index,j in enumerate(img_annotations):
    #print(j)
    height = metadata_df.loc[id_to_file_name[str(j["image_id"])],"height"]
    #print(height)
    width = metadata_df.loc[id_to_file_name[str(j["image_id"])],"width"]
    #print(width)
    #print(j["bbox"])
    if j["bbox"][2]>1 and j["bbox"][3]>1: 
        [xmin,ymin,xmax,ymax]=list(convert_bbox_to_albumentations((j["bbox"][0],j["bbox"][1],j["bbox"][2],j["bbox"][3]),source_format="coco",check_validity=True,rows = height,cols = width))
    
        dict_annotations[index] = {"file_name":id_to_file_name[str(j["image_id"])],
                                      "xmin":xmin,
                                      "ymin":ymin,
                                      "xmax":xmax,
                                     "ymax": ymax}

In [31]:
len(dict_annotations)

11004

In [32]:
annotations_df = pd.DataFrame(dict_annotations).T

In [33]:
annotations_df.head()

,file_name,xmin,ymin,xmax,ymax
0,000000532481.jpg,0.391906,0.394977,0.501453,0.547277
1,000000532481.jpg,0.446172,0.869859,0.465031,0.914953
2,000000458755.jpg,0.107859,0.0786458,0.901688,0.986521
3,000000458755.jpg,0.885938,0.00264583,1,0.183604
4,000000458755.jpg,0.921906,0.191021,0.999437,0.39325


In [34]:
os.chdir("/home/sergio/Documents/COCODataset")
annotations_df.to_csv("COCO_annotations_val.csv",index=True)

In [ ]:
augmented["bboxes"]=convert_bboxes_from_albumentations(augmented["bboxes"],
target_format="pascal_voc",
rows=image.shape[0],
cols=image.shape[1],
check_validity=False)